## Import libraries

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import google.generativeai as genai
from google.colab import userdata
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
import os
import IPython
import time
import re
import shutil
import glob
import numpy as np

## Gemini configuration

In [ ]:
# API key
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

genai.configure(api_key = GOOGLE_API_KEY)

In [ ]:
system_instruction = "You are a sustainability expert consultant. You must respond with facts."

In [ ]:
model = genai.GenerativeModel('gemini-1.5-flash-002',system_instruction=system_instruction) # gemini-1.5-flash # gemini-1.5-pro # gemini-pro # gemini-1.5-flash-002

## Adjust model's temperature, top_p, top_k (0 for more deterministic answers)

In [ ]:
generation_config = genai.GenerationConfig(top_p = 0, temperature = 0, top_k= 1)

## Safety settings

In [ ]:
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

## Mount drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Base path
base_path = "/content/drive/My Drive"

# Define the folder path in your Google Drive
general_path = os.path.join(base_path,"TrabajoWater")

# Check if the folder exists, if not, create it
if not os.path.exists(general_path):
    os.makedirs(general_path)
    print(f"Created path {general_path}")

## Response folder

In [ ]:
# The SDGs predicted in the previous part
sdg_predictions = os.path.join(general_path,"Raw_Results_Flash_002","Prompt_SDG_Allocation_Water_SDG1")

In [ ]:
# The folder where the new responses will be saved.
interaction_water_folder = os.path.join(general_path,"Raw_Results_Flash_002","Prompt_SDG_InteractionWithWater_SDG1")
if not os.path.exists(interaction_water_folder):
    os.makedirs(interaction_water_folder)
    print(f"Created path {interaction_water_folder}")

## Excel with all texts

In [ ]:
path_excel = os.path.join(general_path,"Excels_Original_Cientificos/Water_SDG1.xlsx") # The Excel that is
print(path_excel)

In [ ]:
df = pd.read_excel(path_excel)
df.columns = df.iloc[0]  # Assign the first row as the header
df = df[1:].reset_index(drop=True)

In [ ]:
df.head()

## Functions

In [ ]:
def extract_sdgs_and_reasons(file_path):
    sdgs = []
    reasons = []

    with open(file_path, "r", encoding="UTF-8") as file:
      content = file.read()

    # Preprocess the content to remove unwanted characters like *, **, and -
    content = content.replace("*", "").replace("**", "")

    # Updated SDG pattern to handle SDG names with commas, multiple spaces, or special characters
    sdg_pattern = r'SDG\s*(\d+)\s*([^\n]+?)\s*Reason:'
    reason_pattern = r'Reason:\s*([^\n]+)'

    # Find all SDG matches
    sdg_matches = re.findall(sdg_pattern, content)

    # Find all Reason matches
    reason_matches = re.findall(reason_pattern, content)

    # Pair SDGs and Reasons
    for i, (sdg_num, sdg_name) in enumerate(sdg_matches):
        sdgs.append(f"SDG{sdg_num} {sdg_name.strip()}")

        # Ensure we have a corresponding reason
        if i < len(reason_matches):
            reasons.append(reason_matches[i].strip())

    if len(sdgs) != len(reasons):
        print(f"Mismatch in SDGs andr easons for file {file_path}.")
        print(content)

    if len(sdgs) == 0:
        print(f"No SDG found for file {file_path}")
        print(content)

    return sdgs, reasons


In [ ]:
def map_sdg(sdg):
    sdg = str(sdg).strip().lower()  # Convert to lowercase and remove any extra spaces

    if any(keyword in sdg for keyword in ['no poverty', 'poverty', 'poor']):
        return '1) No Poverty'
    elif any(keyword in sdg for keyword in ['zero hunger', 'hunger']):
        return '2) Zero Hunger'
    elif any(keyword in sdg for keyword in ['good health and well-being', 'healthy', 'health', 'aids']):
        return '3) Good Health and Well-being'
    elif 'quality education' in sdg:
        return '4) Quality Education'
    elif any(keyword in sdg for keyword in ['gender equality', 'women']):
        return '5) Gender Equality'
    elif any(keyword in sdg for keyword in ['clean water and sanitation', 'chemicals', 'water and sanitation']):
        return '6) Clean Water and Sanitation'
    elif any(keyword in sdg for keyword in ['affordable and clean energy', 'energy']):
        return '7) Affordable and Clean Energy'
    elif any(keyword in sdg for keyword in ['decent work and economic growth', 'economic growth']):
        return '8) Decent Work and Economic Growth'
    elif any(keyword in sdg for keyword in ['industry', 'innovation', 'infrastructure']):
        return '9) Industry, Innovation and Infrastructure'
    elif 'reduced inequalities' in sdg or 'inequality' in sdg or 'reduce inequalities' in sdg:
        return '10) Reduced Inequalities'
    elif any(keyword in sdg for keyword in ['sustainable cities', 'cities']):
        return '11) Sustainable Cities and Communities'
    elif any(keyword in sdg for keyword in ['responsible consumption', 'consumption']):
        return '12) Responsible Consumption and Production'
    elif any(keyword in sdg for keyword in ['climate action', 'climate change']):
        return '13) Climate Action'
    elif any(keyword in sdg for keyword in ['life below water', 'oceans', 'life under water']):
        return '14) Life Below Water'
    elif any(keyword in sdg for keyword in ['life on land', 'forest', 'land and soil']):
        return '15) Life on Land'
    elif any(keyword in sdg for keyword in ['peace', 'justice', 'strong institutions','rule of law']):
        return '16) Peace, Justice and Strong Institutions'
    elif any(keyword in sdg for keyword in ['partnerships', 'partnership','sustainable development goals']):
        return '17) Partnerships for the Goals'
    elif any(keyword in sdg for keyword in ['no sdg', 'nan', 'n/a', 'not applicable']):
        return "No SDG"
    elif any(keyword in Link for keyword in ['Synergy']):
        return 'Synergy'
    elif any(keyword in sdg for keyword in ['Trade-off']):
        return 'Trade-off'
    else:
        return sdg


## Loop through Excel

In [ ]:
# Define the number of samples
start_index = 0

# Define the starting index for the second batch
end_index = 4875

# Loop through the indexes of the DataFrame from 0 to 20
for index in range(start_index, end_index):
    # Cada texto del excel
    print("%")
    print(index)
    paragraph = df.loc[index, 'FullText']

    # Old answers
    sdg_prediction_path = os.path.join(sdg_predictions, f"{index}.txt")

    if not os.path.exists(sdg_prediction_path):
        print(f"Skipping {sdg_prediction_path}...")
        continue

    sdgs, reasons = extract_sdgs_and_reasons(sdg_prediction_path)
    relevant_sdgs = []
    for sdg in sdgs:
            mapped_sdg = map_sdg(sdg)
            if mapped_sdg != 'Wrong' and mapped_sdg != 'No SDG' and mapped_sdg != '6) Clean Water and Sanitation':
                relevant_sdgs.append(mapped_sdg)

    if len(relevant_sdgs) <= 0:
      print(f"Skipping file {sdg_prediction_path}, no relevant sdgs...")
      continue

    # Where to save the new answer
    destination_path = os.path.join(interaction_water_folder, f"{index}.txt")
    if os.path.exists(destination_path):
        print(f"Skipping {destination_path} because it already exists...")
        continue

    prompt = f"""
    You are provided with the content of a scientific article about Water. Your task is to analyze the relationship between each predicted SDG with SDG 6) Clean Water and Sanitation. Exclude any references, acknowledgments, titles, and authors in your decision-making, and focus exclusively on the article’s main content.

    **Instructions**

    You must identify whether the link would result in a synergy, a trade-off, or it is a neutral link:

      - Synergy: achieving the SDG through the measures discussed in the article would benefit SDG 6) Clean Water and Sanitation
      - Trade-off: achieving the SDG through the measures discussed in the article would put pressure or harm SDG 6) Clean Water and Sanitation
      - Neutral: achieving the SDG through the measures discussed in the article would not have a significant effect or the link is not clear between the SDG and SDG 6) Clean Water and Sanitation

     Write an exact sentence from the article that led you to decide whether there is synergy, trade-off or neutral.

    **Original article**

    Article: "{paragraph}"

    **Output Format:**
    """

    for sdg in relevant_sdgs:
      if sdg != "6) Clean Water and Sanitation":
        prompt += f"""
              - SDG Pair: SDG {sdg} - SDG 6) Clean Water and Sanitation
                - Relationship: [Synergy/Trade-off/Neutral]
                - Explanation: [Reason how each measure impacts SDG 6) Clean Water and Sanitation with references from the text and reasoning the possible outcomes according to the achievement of SDG {sdg} in the context of the article]
        """

    print(prompt)

    try:
        response = model.generate_content(prompt, safety_settings=safety_settings, generation_config=generation_config)
        print(response.text)

        with open(destination_path, 'w') as file:  # Asegúrate de abrir el archivo aquí
          file.write(response.text)

    except Exception as e:
        if '500' in str(e):
            print("=========================================== Exception 500 =========================================== ")
        elif '429' in str(e):  # si sale más de 20 veces seguidas, cambiar de clave o esperar al día siguiente
            print("=========================================== Exception 429 =========================================== ")
            time.sleep(5)
        elif 'Read timed out' in str(e):  # Reiniciar sesión y ejecutar todo
            print("=========================================== Exception 600 =========================================== ")
        else:
            print(f"Skipping due to error: {e}")


## Extract responses

In [ ]:
txt_files = glob.glob(os.path.join(interaction_water_folder, '*.txt'))

csv = df.copy()

# Define a function to extract numerical part from the filename
def extract_number(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group()) if match else float('inf')

# Sort files based on the numerical part
sorted_files = sorted(txt_files, key=lambda f: extract_number(os.path.basename(f)))
print(sorted_files)

In [ ]:
def extract_info_from_file(file_path):

    results = []

    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    text = text.replace("**","")

    # ************************************************************************ #

    pattern = re.compile(
    r"SDG Pair:\s+SDG\s*(\d+)\s*\)\s*(.*?)\s*-\s*SDG\s*(\d+)\s*\)\s*(.*?)\n"
    r"\s*- Relationship:\s*(.*?)\n"
    r"\s*- Explanation:\s*(.*?)(?=\n\s*- SDG Pair|\Z)",
    re.DOTALL
    )

    # Find all matches in the text
    matches = pattern.findall(text)

    # Check if any matches were found
    if not matches:
        print(f"No matches found\n{text}")
        return results

    # Iterate through the matches and build the result list
    for match in matches:
        sdg_a = f"SDG{match[0]} {match[1].strip()}"
        if match[2]:  # If second SDG is captured
            sdg_b = f"SDG{match[2]} {match[3].strip()}"
        else:  # If no second SDG is present
            sdg_b = f"SDG{match[0]} {match[1].strip()}"  # Duplicate SDG with empty text for consistency


        relationship = match[4].strip()

        explanation = match[5].strip()

        results.append((sdg_a, sdg_b, relationship, explanation))



    # ************************************************************************ #


    return results

In [ ]:
def process_files_and_update_dataframe(sorted_files, original_df, response_directory_ST):

    new_rows = []
    current_index = 0  # Keep track of the expected index
    suma = 0

    for file in sorted_files:
        # Extract index from filename
        index = extract_number(os.path.basename(file))  # Assuming you have a function to extract the number
        print("/" * 50)
        print(index)

        file_path = os.path.join(response_directory_ST, f'{index}.txt')
        if not os.path.exists(file_path):
            # Create a new row with the original row's data
            row = original_df.loc[index].copy()
            new_rows.append(row)
            continue

        # Extract SDG-related information from the file
        all_pairs_info = extract_info_from_file(file_path)

        # Iterate through extracted pairs
        if all_pairs_info:
          for sdg_a, sdg_b, relationship, explanation in all_pairs_info:
              # Add a new row with SDG information
              row = original_df.loc[index].copy()  # Get the existing row for the current index
              row['SDG A'] = sdg_a
              row['SDG B'] = sdg_b
              row['Link'] = relationship
              row['Explanation'] = explanation
              new_rows.append(row)
              current_index += 1
        else:
            # Create a new row with the original row's data
            print("No prints found...")
            print("*" * 50)
            print(f"Index: {index}")
            row = original_df.loc[index].copy()
            new_rows.append(row)
            suma += 1

    # Create a new dataframe from the rows collected
    updated_df = pd.DataFrame(new_rows)

    # Return the new dataframe with SDG information
    print(f"There are {suma} wrong files")
    return updated_df

In [ ]:
df_updated = process_files_and_update_dataframe(sorted_files, csv, interaction_water_folder)

In [ ]:
df_updated.head()

In [ ]:
df_updated.to_excel(os.path.join(general_path,"SDG1_interaction.xlsx"))

In [ ]:
txt_files = glob.glob(os.path.join(sdg_predictions, '*.txt'))

# Sort files based on the numerical part
sorted_files_2 = sorted(txt_files, key=lambda f: extract_number(os.path.basename(f)))
print(sorted_files_2)

In [ ]:
# Load the main CSV file
main_csv = df_updated.copy()

# Initialize a list to store the final data with columns from the main CSV file
final_data = []

# Initialize a list of available indices based on the files we have
available_indices = [extract_number(os.path.basename(file)) for file in sorted_files_2]

# Process each file in sorted order
for file in sorted_files_2:
    # Extract the index or identifier for this file
    file_index = extract_number(os.path.basename(file))
    sdgs, reasons = extract_sdgs_and_reasons(file)  # Extract SDGs and reasons

    # Extract relevant SDGs based on mapping
    relevant_sdgs = []
    for sdg in sdgs:
        mapped_sdg = map_sdg(sdg)
        if mapped_sdg != 'Wrong' and mapped_sdg != 'No SDG' and mapped_sdg != '6) Clean Water and Sanitation':
            relevant_sdgs.append(mapped_sdg)

    # Find the row in main_csv that corresponds to this file's index
    if file_index < len(main_csv):
        original_row = main_csv.iloc[file_index].to_dict()
    else:
        # If index not found, skip to next file
        continue

    # Add the original paragraph number
    original_row['Paragraph number'] = file_index

    # Case 1: If only one or no relevant SDG is allocated
    #if len(relevant_sdgs) <= 1:
    #    final_data.append({**original_row, 'SDG A': 'Only 1 SDG Allocated', 'SDG B': 'Only 1 SDG Allocated'})
    #    continue

    # Case 2: Generate all combinations of SDG pairs for this index and add to the data
    for sdg in relevant_sdgs:
        final_data.append({**original_row, 'SDG A': sdg, 'SDG B': '6) Clean Water and Sanitation'})

# Process the indices in main_csv that are not available in sorted_files
for idx, row in main_csv.iterrows():
    if idx not in available_indices:
        row_data = row.to_dict()
        row_data.update({
            'SDG A': 'Blocked SDG Allocation',
            'SDG B': 'Blocked SDG Allocation',
            'Paragraph number': idx
        })
        final_data.append(row_data)

# Convert the final data list to a DataFrame
df_sdg_pairs = pd.DataFrame(final_data)

df_sdg_pairs.insert(0, 'Paragraph number', df_sdg_pairs.pop('Paragraph number'))

In [ ]:
df_sdg_pairs.head()

In [ ]:
def get_sdg_number(sdg_full_name):
    sdg_map = {
        '1) No Poverty': '1',
        '2) Zero Hunger': '2',
        '3) Good Health and Well-being': '3',
        '4) Quality Education': '4',
        '5) Gender Equality': '5',
        '6) Clean Water and Sanitation': '6',
        '7) Affordable and Clean Energy': '7',
        '8) Decent Work and Economic Growth': '8',
        '9) Industry, Innovation and Infrastructure': '9',
        '10) Reduced Inequalities': '10',
        '11) Sustainable Cities and Communities': '11',
        '12) Responsible Consumption and Production': '12',
        '13) Climate Action': '13',
        '14) Life Below Water': '14',
        '15) Life on Land': '15',
        '16) Peace, Justice and Strong Institutions': '16',
        '17) Partnerships for the Goals': '17',
        'No SDG': 'No SDG'
    }

    return sdg_map.get(sdg_full_name, sdg_full_name)

In [ ]:
# Apply the mapping function to both columns
df_sdg_pairs['SDG A c'] = df_sdg_pairs['SDG A'].apply(get_sdg_number)
df_sdg_pairs['SDG B c'] = df_sdg_pairs['SDG B'].apply(get_sdg_number)

In [ ]:
# Print the unique values of 'Link' along with their counts
link_counts = df_sdg_pairs['Link'].value_counts()

print(link_counts)

In [ ]:
# Step 3: Count occurrences of each pair using groupby
pair_counts = df_sdg_pairs.groupby(['SDG A c', 'SDG B c']).size().reset_index(name='count')

# Step 4: Create a pivot table
pivot_table = pair_counts.pivot(index='SDG A c', columns='SDG B c', values='count').fillna(0)

# Step 5: Sort the index and columns numerically
pivot_table.index = pd.to_numeric(pivot_table.index, errors='coerce')
pivot_table.columns = pd.to_numeric(pivot_table.columns, errors='coerce')
pivot_table = pivot_table.sort_index().sort_index(axis=1)

# Step 6: Plotting the heatmap
plt.figure(figsize=(8, 6))
heatmap = sns.heatmap(pivot_table, annot=True, cmap='Reds', fmt=".0f", linewidths=.5, cbar_kws={"orientation": "vertical"},cbar=False)

# Adjust tick positions to be centered
plt.xticks(ticks=np.arange(len(pivot_table.columns)) + 0.5, labels=pivot_table.columns.astype(str), rotation=0)
plt.yticks(ticks=np.arange(len(pivot_table.index)) + 0.5, labels=pivot_table.index.astype(str), rotation=0)

# Title and labels
plt.title('All connections - Gemini - Standard')
plt.xlabel('Connected SDG')
plt.ylabel('Base SDG')

plt.show()

In [ ]:
# Step 1: Filter the DataFrame where 'Link' is 'Synergy'
df_out = df_sdg_pairs[df_sdg_pairs['Link'] == 'Synergy']

# Step 3: Count occurrences of each pair using groupby
pair_counts = df_out.groupby(['SDG A c', 'SDG B c']).size().reset_index(name='count')

# Step 4: Create a pivot table
pivot_table = pair_counts.pivot(index='SDG A c', columns='SDG B c', values='count').fillna(0)

# Step 5: Sort the index and columns numerically
pivot_table.index = pd.to_numeric(pivot_table.index, errors='coerce')
pivot_table.columns = pd.to_numeric(pivot_table.columns, errors='coerce')
pivot_table = pivot_table.sort_index().sort_index(axis=1)

# Step 6: Plotting the heatmap
plt.figure(figsize=(8, 6))
heatmap = sns.heatmap(pivot_table, annot=True, cmap='Reds', fmt=".0f", linewidths=.5, cbar_kws={"orientation": "vertical"},cbar=False)

# Adjust tick positions to be centered
plt.xticks(ticks=np.arange(len(pivot_table.columns)) + 0.5, labels=pivot_table.columns.astype(str), rotation=0)
plt.yticks(ticks=np.arange(len(pivot_table.index)) + 0.5, labels=pivot_table.index.astype(str), rotation=0)

# Title and labels
plt.title('Synergy connections - Gemini ')
plt.xlabel('Connected SDG')
plt.ylabel('Base SDG')

plt.show()

In [ ]:
# Step 1: Filter the DataFrame where 'Link' is 'Synergy'
df_out = df_sdg_pairs[df_sdg_pairs['Link'] == 'Trade-off']

# Step 3: Count occurrences of each pair using groupby
pair_counts = df_out.groupby(['SDG A c', 'SDG B c']).size().reset_index(name='count')

# Step 4: Create a pivot table
pivot_table = pair_counts.pivot(index='SDG A c', columns='SDG B c', values='count').fillna(0)

# Step 5: Sort the index and columns numerically
pivot_table.index = pd.to_numeric(pivot_table.index, errors='coerce')
pivot_table.columns = pd.to_numeric(pivot_table.columns, errors='coerce')
pivot_table = pivot_table.sort_index().sort_index(axis=1)

# Step 6: Plotting the heatmap
plt.figure(figsize=(8, 6))
heatmap = sns.heatmap(pivot_table, annot=True, cmap='Reds', fmt=".0f", linewidths=.5, cbar_kws={"orientation": "vertical"},cbar=False)

# Adjust tick positions to be centered
plt.xticks(ticks=np.arange(len(pivot_table.columns)) + 0.5, labels=pivot_table.columns.astype(str), rotation=0)
plt.yticks(ticks=np.arange(len(pivot_table.index)) + 0.5, labels=pivot_table.index.astype(str), rotation=0)

# Title and labels
plt.title('Trade-off connections - Gemini ')
plt.xlabel('Connected SDG')
plt.ylabel('Base SDG')

plt.show()

In [ ]:
# Step 1: Filter the DataFrame where 'Link' is 'Synergy'
df_out = df_sdg_pairs[df_sdg_pairs['Link'] == 'Neutral']

# Step 3: Count occurrences of each pair using groupby
pair_counts = df_out.groupby(['SDG A c', 'SDG B c']).size().reset_index(name='count')

# Step 4: Create a pivot table
pivot_table = pair_counts.pivot(index='SDG A c', columns='SDG B c', values='count').fillna(0)

# Step 5: Sort the index and columns numerically
pivot_table.index = pd.to_numeric(pivot_table.index, errors='coerce')
pivot_table.columns = pd.to_numeric(pivot_table.columns, errors='coerce')
pivot_table = pivot_table.sort_index().sort_index(axis=1)

# Step 6: Plotting the heatmap
plt.figure(figsize=(8, 6))
heatmap = sns.heatmap(pivot_table, annot=True, cmap='Reds', fmt=".0f", linewidths=.5, cbar_kws={"orientation": "vertical"},cbar=False)

# Adjust tick positions to be centered
plt.xticks(ticks=np.arange(len(pivot_table.columns)) + 0.5, labels=pivot_table.columns.astype(str), rotation=0)
plt.yticks(ticks=np.arange(len(pivot_table.index)) + 0.5, labels=pivot_table.index.astype(str), rotation=0)

# Title and labels
plt.title('Neutral connections - Gemini ')
plt.xlabel('Connected SDG')
plt.ylabel('Base SDG')

plt.show()